# Example BSDF in Python

In [1]:
import os
import bsdf
filename = os.path.expanduser('~/foo.bsdf')

## Simple use
Here, we define some data, encode it, and decode back.

In [2]:
data = ['hello', 3, 4, None, {'foo': [1, 2], 'bar': b'xxxxx'}]
bb = bsdf.encode(data)
bb

b'BSDF\x02\x01l\x05s\x05helloh\x03\x00h\x04\x00vm\x02\x03fool\x02h\x01\x00h\x02\x00\x03barb\x05\x05\x05\x00\x00\x01\x00xxxxx'

In [3]:
bsdf.decode(bb)

['hello', 3, 4, None, {'bar': b'xxxxx', 'foo': [1, 2]}]

The above encodes/decodes to/from bytes. We can also use file objects. `bsdf.save()` and `bsdf.load()` also support file names.

In [4]:
bsdf.save(filename, data)
bsdf.load(filename)

['hello', 3, 4, None, {'bar': b'xxxxx', 'foo': [1, 2]}]

## Custom extensions
Extensions are easy to write.

In [5]:
# For the sake of example, let's write an extension that encodes complex numbers.
# (Spoiler: this is the actual code for the standard complex extention)
class ComplexExtension(bsdf.Extension):

    name = 'c'
    cls = complex

    def encode(self, s, v):
        return (v.real, v.imag)

    def decode(self, s, v):
        return complex(v[0], v[1])
    

In [6]:
bb = bsdf.encode(3+4j, [ComplexExtension])
print(bb)
bsdf.decode(bb, [ComplexExtension])

b'BSDF\x02\x01L\x01c\x02d\x00\x00\x00\x00\x00\x00\x08@d\x00\x00\x00\x00\x00\x00\x10@'


(3+4j)

In [7]:
# This would work too, because it is a standard extension
# bsdf.decode(bb)
# But this would not
# bsdf.decode(bb, [])

## Use of serializer class
The serializer class is recommended when performance matters. It also defines what options and extensions to use; using the `s` will use compression by default and supports complex numbers. The serializer class has methods `save()`, `load()`, `encode()`, `decode()`, and methods to add/remove extensions. Note that `save()` and `load()` need file objects (not file names).


In [8]:
s = bsdf.BsdfSerializer([ComplexExtension], compression=1)
s

In [9]:
len(s.encode(b'x'*1000))

54

## Streaming
BSDF support streamed wirting and reading of lists though the `ListStream` class.

In [10]:
ls = bsdf.ListStream()
ls

The stream object is used as an element in a BSDF structure. It must be the very last element. In this case
we simply use it as the root element.

In [11]:
f = open(filename, 'wb')
bsdf.save(f, ls)
f.tell()  # only the BSDF header is written and the identifier for the streaming list

16

Now we can write whatever we want (as long as its BSDF serializable)!

In [12]:
for i in range(4, 9):
    ls.append('a' * i)
for i in range(4, 9):
    ls.append(i)
ls.append('the list can of course contain anything')
ls.append([False, None, 'the end'])

That should be enough.

In [13]:
f.close()
# ls.append(3) This raises IOError; cannot write to a close file

We can load the written data at once:

In [14]:
bsdf.load(filename)

['aaaa',
 'aaaaa',
 'aaaaaa',
 'aaaaaaa',
 'aaaaaaaa',
 4,
 5,
 6,
 7,
 8,
 'the list can of course contain anything',
 [False, None, 'the end']]

Or we can load in streaming mode:

In [15]:
f = open(filename, 'rb')
ls = bsdf.load(f, load_streaming=True)
ls

At this point, only the BSDF header and the list identifier have been read:

In [16]:
f.tell()

16

Data is read on demand:

In [17]:
print(ls.next())
f.tell()

aaaa


22

In [18]:
for item in ls:
    print(item)
f.close()

aaaaa
aaaaaa
aaaaaaa
aaaaaaaa
4
5
6
7
8
the list can of course contain anything
[False, None, 'the end']


## Appending data to a stream
Since the stream above is not closed, we can add data to it. The "simple" way is by appending bytes to the file, though this requires knowledge of the BSDF format.

In [19]:
with open(filename, 'ab') as f:
    f.write(b'v')  # None (void)
    f.write(b'h*\x00')  # The number 42
bsdf.load(filename)

['aaaa',
 'aaaaa',
 'aaaaaa',
 'aaaaaaa',
 'aaaaaaaa',
 4,
 5,
 6,
 7,
 8,
 'the list can of course contain anything',
 [False, None, 'the end'],
 None,
 42]

But we can also use the BSDF API ...

In [20]:
with open(filename, 'r+b') as f:  # Note the r+ mode
    ls = bsdf.load(f, load_streaming=True)
    for i in ls:  # This looks funny, but we need to move to the end of the stream
        pass
    ls.append([3, 4, 5])
    ls.append('that should be it')    

In [21]:
bsdf.load(filename)

['aaaa',
 'aaaaa',
 'aaaaaa',
 'aaaaaaa',
 'aaaaaaaa',
 4,
 5,
 6,
 7,
 8,
 'the list can of course contain anything',
 [False, None, 'the end'],
 None,
 42,
 [3, 4, 5],
 'that should be it']

## Modifying binary blobs
Start by writing a file that contains bytes. 

In [22]:
bsdf.save(filename, [3, 4, b'xxyyzz'])

Open in r+ mode to be able to modify the data.

In [23]:
with open(filename, 'r+b') as f:
    blob = bsdf.load(f, lazy_blob=True)[2]
    blob.seek(0)  # we need to move to the start of the blob
    print(blob.read(2))  # xx
    # Change the last two bytes from zz to aa
    blob.seek(4)
    blob.write(b'aa')
    # Update the checksum, in case there is one
    blob.update_checksum()

b'xx'


And check the data ...

In [24]:
bsdf.load(filename)

[3, 4, b'xxyyaa']